In [1]:
from IPython.display import clear_output

In [2]:
!pip install langchain langchain_openai
clear_output()

In [39]:
import os
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import Annotated, List

In [40]:
# Registramos la API
with open("/content/api_key_openai.txt") as archivo:
  apikey = archivo.read()

os.environ["OPENAI_API_KEY"] =apikey

## Creacion de herramientas

LangChain admite la creación de herramientas desde:
- Funciones @tool
- Runnables de LangChain
- Subclase de BaseTool: este es el método más flexible y proporciona el mayor grado de control, a costa de más esfuerzo y código.

### 1 Herramienta basada en @tool




In [5]:
@tool
def add(a: int, b: int) -> int:
    """Adicionar dos numeros"""
    a= int(a)
    b= int(b)
    return a + b

In [8]:
add.args_schema.model_json_schema()

{'description': 'Adicionar dos numeros',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'add',
 'type': 'object'}

In [9]:
# Inspeccionamos los atributos
print("Nombre: ",add.name)
print("Descripcion: ",add.description)
print("Argumentos: ",add.args)
print("Retorno: ",add.return_direct)

Nombre:  add
Descripcion:  Adicionar dos numeros
Argumentos:  {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
Retorno:  False


### Creacion con @tool + anotaciones




In [10]:
@tool
def multiply(
    a: Annotated[int, "Primero operador de multiplicacion"],
    b: Annotated[int, "Segundo operador de multiplicacion"],) -> int:
    """Multiplicar dos numeros"""
    return a * b

In [12]:
multiply.args_schema.model_json_schema()

{'description': 'Multiplicar dos numeros',
 'properties': {'a': {'description': 'Primero operador de multiplicacion',
   'title': 'A',
   'type': 'integer'},
  'b': {'description': 'Segundo operador de multiplicacion',
   'title': 'B',
   'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiply',
 'type': 'object'}

In [13]:
# Inspeccionamos los atributos
print("Nombre: ",multiply.name)
print("Descripcion: ",multiply.description)
print("Argumentos: ",multiply.args)
print("Retorno: ",multiply.return_direct)

Nombre:  multiply
Descripcion:  Multiplicar dos numeros
Argumentos:  {'a': {'description': 'Primero operador de multiplicacion', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Segundo operador de multiplicacion', 'title': 'B', 'type': 'integer'}}
Retorno:  False


### Personalizar @tool el nombre y argumentos json mediante el decorador

In [16]:
from pydantic import BaseModel, Field

class SubInput(BaseModel):
    a: int = Field(description="Primer numero")
    b: int = Field(description="Segundo numero")

@tool("sub-tool", args_schema=SubInput, return_direct=True)
def sub(a: int, b: int) -> int:
    """resta dos numeros"""
    return a - b


In [18]:
sub.args_schema.model_json_schema()

{'properties': {'a': {'description': 'Primer numero',
   'title': 'A',
   'type': 'integer'},
  'b': {'description': 'Segundo numero', 'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'SubInput',
 'type': 'object'}

In [19]:
# Inspeccionamos los atributos
print("Nombre: ",sub.name)
print("Descripcion: ",sub.description)
print("Argumentos: ",sub.args)
print("Retorno: ",sub.return_direct)

Nombre:  sub-tool
Descripcion:  resta dos numeros
Argumentos:  {'a': {'description': 'Primer numero', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Segundo numero', 'title': 'B', 'type': 'integer'}}
Retorno:  True


### 2 Herramienta basada en runnables

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Elige el modelo LLM
llm = ChatOpenAI()

class BinarizerInput(BaseModel):
    input_number: int = Field(description="Número que se va a convertir a binario")

prompt = ChatPromptTemplate.from_messages(
    [("system", "convierte el siguiente número a binario: {input_number}")]
)

chain = prompt | llm | StrOutputParser()


binarizador_tool = chain.as_tool(
    name="Binarizador", description="Convierte cualquier numero en binario",
    args_schema=BinarizerInput
)


In [23]:
binarizador_tool.args_schema.model_json_schema()

{'properties': {'input_number': {'description': 'Número que se va a convertir a binario',
   'title': 'Input Number',
   'type': 'integer'}},
 'required': ['input_number'],
 'title': 'BinarizerInput',
 'type': 'object'}

In [24]:
# Inspeccionamos los atributos
print("Nombre: ",binarizador_tool.name)
print("Descripcion: ",binarizador_tool.description)
print("Argumentos: ",binarizador_tool.args)
print("Retorno: ",binarizador_tool.return_direct)

Nombre:  Binarizador
Descripcion:  Convierte cualquier numero en binario
Argumentos:  {'input_number': {'description': 'Número que se va a convertir a binario', 'title': 'Input Number', 'type': 'integer'}}
Retorno:  False


### 3 Herramienta basada en la subclase BaseTool

In [27]:
from typing import Optional, Type

from pydantic import BaseModel
from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain_core.tools import BaseTool


class PowInput(BaseModel):
    a: int = Field(description="Numero base")
    b: int = Field(description="Numero potencia")


class PowCustom(BaseTool):
    name: str  = "PotenciaPersonalizado"
    description: str = "Permite calcular la potencia de un numero base y su potencia"
    args_schema: Type[BaseModel] = PowInput
    # Indica que el resultado de la herramienta debe ser devuelto directamente
    return_direct: bool = True

    # Método que ejecuta la herramienta de manera sincrónica
    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Usa la herramienta."""
        return a ** b
    # Método que ejecuta la herramienta de manera asincrónica
    async def _arun(
        self,
        a: int,
        b: int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        # Si el cálculo es ligero, puedes delegar la tarea a la implementación sincrónica como se muestra abajo.
        # Si el cálculo sincrónico es costoso, deberías eliminar todo el método `_arun`.
        # LangChain proporcionará automáticamente una mejor implementación que iniciará la tarea en un hilo separado,
        # asegurándose de que no bloquee otro código asincrónico.

        return self._run(a, b, run_manager=run_manager.get_sync())
toolPowCustom = PowCustom()

In [29]:
toolPowCustom.args_schema.model_json_schema()

{'properties': {'a': {'description': 'Numero base',
   'title': 'A',
   'type': 'integer'},
  'b': {'description': 'Numero potencia', 'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'PowInput',
 'type': 'object'}

In [30]:
# Inspeccionamos los atributos
print("Nombre: ",toolPowCustom.name)
print("Descripcion: ",toolPowCustom.description)
print("Argumentos: ",toolPowCustom.args)
print("Retorno: ",toolPowCustom.return_direct)

Nombre:  PotenciaPersonalizado
Descripcion:  Permite calcular la potencia de un numero base y su potencia
Argumentos:  {'a': {'description': 'Numero base', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Numero potencia', 'title': 'B', 'type': 'integer'}}
Retorno:  True


In [31]:
# Configurar el prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """Eres un asistente matematico, usa tus herramientas para responder preguntas.
        Si no tienes una herramienta explícita para responder la pregunta, responde con tu propio conocimiento.

        Retorna solo la respuesta. por ejemplo:

        Human: ¿Cuánto es 1+1?
        AI: 2
        """),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [32]:
# Elige el modelo LLM
llm = ChatOpenAI()
# Configurar el conjunto de herramientas
toolkit = [add, multiply, sub, binarizador_tool, toolPowCustom]

In [33]:
# Construir el agente
agent = create_openai_tools_agent(
    llm=llm,
    tools=toolkit,
    prompt=prompt
)

# Crear el ejecutor del agente
agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True,return_intermediate_steps=True)

### Ejemplo de uso de herramientas

In [41]:
# Ejecutar una consulta
result = agent_executor.invoke({"input": "¿Cuánto es el cubo de 3 más la suma de 4 y 5?"})

# Imprimir la salida
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `PotenciaPersonalizado` with `{'a': 3, 'b': 3}`


27

Invoking: `add` with `{'a': 4, 'b': 5}`


9
Invoking: `add` with `{'a': 27, 'b': 9}`


36El cubo de 3 más la suma de 4 y 5 es igual a 36.

> Finished chain.
El cubo de 3 más la suma de 4 y 5 es igual a 36.


In [42]:
# Ejecutar una consulta
result = agent_executor.invoke({"input": "¿indica cuanto es el valor binario de 96?"})

# Imprimir la salida
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `Binarizador` with `{'input_number': 96}`


El número 96 en binario es 1100000.El valor binario de 96 es 1100000.

> Finished chain.
El valor binario de 96 es 1100000.


In [43]:
# Ejecutar una consulta
result = agent_executor.invoke({"input": "¿Cual es la potencia  de 3 elevado a 4 y la resta de 9 y 7?"})

# Imprimir la salida
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `PotenciaPersonalizado` with `{'a': 3, 'b': 4}`


81

Invoking: `sub-tool` with `{'a': 9, 'b': 7}`


2
La potencia de 3 elevado a 4 es 81 y la resta de 9 y 7 es 2.

> Finished chain.
La potencia de 3 elevado a 4 es 81 y la resta de 9 y 7 es 2.


### Visuzalizacion de pasos intermedios

In [44]:
# Imprimir directamente los pasos intermedios y la respuesta final
print("Pasos intermedios:")
for step in result['intermediate_steps']:
    action, observation = step
    print(f"Acción: {action.tool}, Input: {action.tool_input}, Resultado: {observation}")


Pasos intermedios:
Acción: PotenciaPersonalizado, Input: {'a': 3, 'b': 4}, Resultado: 81
Acción: sub-tool, Input: {'a': 9, 'b': 7}, Resultado: 2


In [38]:
# Este codigo se basa en la documentacion oficial version 0.2 y 0.3 langchain